In [1]:
from tensorflow.keras import backend as K
import os
import numpy as np
import random
from tensorflow.keras.initializers import glorot_uniform
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.layers import Input, Concatenate, Conv2D, MaxPooling2D, UpSampling2D, Dropout,BatchNormalization,ZeroPadding2D,add, Flatten,Activation,AveragePooling2D,Dense
from tensorflow.keras.optimizers import Adam,SGD,RMSprop
from tensorflow.keras.callbacks import ModelCheckpoint, LearningRateScheduler,CSVLogger,ReduceLROnPlateau
from tensorflow.keras.layers import Lambda
from tensorflow.keras.preprocessing.image import img_to_array,ImageDataGenerator
from tensorflow.keras import backend as K
from tensorflow.keras.utils import plot_model,to_categorical
import h5py
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from PIL import Image, ImageEnhance, ImageOps, ImageFile
import time
from all_model import all_model
from utils import *
# from historycal import LossHistory
# os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [12]:
# Initialize mwutils
!pip install modelwhaleutils

from mwutils.keras import MWCustomCallback
from mwutils.run import Run           # 导入训练记录所需模块 

r = Run(name = 'time_v2',
        user_id = "62553fd0d363d500175e53f3",
        lab_id = "62569aa5231f0d0017312c82",
        org_id = "6228c286f6d86400178f93a4",
        remote_path="http://10.82.11.57:30001/api/runs"
        )


remotepath is http://10.82.11.57:30001/api/runs


In [3]:
hdf5_path = "/home/nmic/input/nas5918/HJtest2-tf1/data/train_5000.hdf5"
fd = h5py.File(hdf5_path)
images,labels=fd['image'],fd['label']
images,labels=np.asarray(fd['image']),np.asarray(fd['label'])
print(images.shape,labels.shape)
fd.close()

# classes = np.unique(labels)  #标签类别
# class_weight = compute_class_weight('balanced', classes, labels.ravel())
# print(class_weight)
# class_weight=[0.27993096,1.59620928,32.11132344,1.81175424,0.842855,31.57588407]

n_label=6 #耕地=0，林地=1，草地=2，水域=3，居民地=4，未利用=5
img,msk = get_normalized_patches(images[:2000],labels[:2000],n_label)

(5000, 256, 256, 3) (5000, 256, 256)
(2000, 256, 256, 3) (2000, 256, 256, 6)


In [4]:
del images,labels
xtrain,xtest,ytrain,ytest=train_test_split(img,msk,test_size=0.2,random_state=42)
del img,msk
print(xtrain.shape,xtest.shape,ytrain.shape,ytest.shape)

(1600, 256, 256, 3) (400, 256, 256, 3) (1600, 256, 256, 6) (400, 256, 256, 6)


In [5]:
plot_func(xtrain[40:, :, :, :], ytrain[40:, :, :, 0])

<Figure size 1800x360 with 20 Axes>

<Figure size 1800x360 with 20 Axes>

In [6]:
batch_size=10
epoch=10
nickname='UNET_VGG'

In [7]:
loss='categorical_crossentropy'#'categorical_crossentropy'#'binary_crossentropy'FocalLoss
optimizer=Adam()#Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)#Adam()#SGD(),Adamax(),Adadelta()R
loss_weights=''
metrics=['accuracy']
input_height=256
input_width=256
nclass=n_label
nchannel=xtrain.shape[-1]
num_train=xtrain.shape[0]
num_val=xtest.shape[0]
ATM=all_model(loss,loss_weights,optimizer,metrics,input_height,input_width,nclass,nchannel)

In [8]:
# model=ATM.UNET_VGG()#FCN8,UNET_VGG,UNET_MINI,SQUEESE_UNET,DEEPLABV3plus,CBRRNET,GFCpsp_xception
modelname='ATM.'+(nickname+'()')
model=eval(modelname)

2022-04-22 10:02:08.878909: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-22 10:02:10.074798: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 112 MB memory:  -> device: 0, name: Tesla V100S-PCIE-32GB, pci bus id: 0000:3b:00.0, compute capability: 7.0
2022-04-22 10:02:10.091593: I tensorflow/stream_executor/cuda/cuda_driver.cc:739] failed to allocate 112.19M (117637120 bytes) from device: CUDA_ERROR_OUT_OF_MEMORY: out of memory


In [14]:
begin_time = time.time()
checkpoint_DIR = "../temp/checkpoint/"
if not os.path.exists(checkpoint_DIR):
    os.makedirs(checkpoint_DIR)
model_checkpoint = ModelCheckpoint(filepath=checkpoint_DIR + nickname + "-{epoch:02d}e-val_loss{val_loss:2f}.hdf5", monitor="val_loss",save_best_only=False,mode='auto')
# csvlogger = CSVLogger(filename=checkpoint_DIR +nickname +'-' +str(epoch) +'-log.csv', separator=',', append=False)
# result=model.fit_generator(G1,verbose=1,steps_per_epoch=num_train/batch_size,epochs=epoch,validation_data=G2,validation_steps=num_val/batch_size,callbacks=[model_checkpoint,histories],shuffle=True )
callBack = MWCustomCallback()
callBack.set_run(r)  
result = model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epoch,verbose=1,shuffle=True,validation_data=( xtest, ytest),callbacks=[model_checkpoint,callBack])

# result = model.fit(xtrain,ytrain,batch_size=batch_size,epochs=epoch,verbose=1,shuffle=True,validation_data=( xtest, ytest),callbacks=[ model_checkpoint, csvlogger])
end_time = time.time()
print('time lapsing {0} s \n'.format(end_time - begin_time))
r.conclude()


2022-04-22 10:10:30.743421: W tensorflow/core/common_runtime/bfc_allocator.cc:462] Allocator (GPU_0_bfc) ran out of memory trying to allocate 1.17GiB (rounded to 1258291200)requested by op _EagerConst
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2022-04-22 10:10:30.743531: I tensorflow/core/common_runtime/bfc_allocator.cc:1010] BFCAllocator dump for GPU_0_bfc
2022-04-22 10:10:30.743570: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (256): 	Total Chunks: 29, Chunks in use: 29. 7.2KiB allocated for chunks. 7.2KiB in use in bin. 2.2KiB client-requested in use in bin.
2022-04-22 10:10:30.743597: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (512): 	Total Chunks: 5, Chunks in use: 5. 2.5KiB allocated for chunks. 2.5KiB in use in bin. 2.5KiB client-requested in use in bin.
2022-04-22 10:10:30.743623: I ten

InternalError: Failed copying input tensor from /job:localhost/replica:0/task:0/device:CPU:0 to /job:localhost/replica:0/task:0/device:GPU:0 in order to run _EagerConst: Dst tensor is not initialized.

 in bin.
2022-04-22 10:10:30.743948: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (2097152): 	Total Chunks: 4, Chunks in use: 3. 9.00MiB allocated for chunks. 6.50MiB in use in bin. 6.50MiB client-requested in use in bin.
2022-04-22 10:10:30.743974: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (4194304): 	Total Chunks: 3, Chunks in use: 2. 13.50MiB allocated for chunks. 9.00MiB in use in bin. 9.00MiB client-requested in use in bin.
2022-04-22 10:10:30.743999: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (8388608): 	Total Chunks: 2, Chunks in use: 1. 18.00MiB allocated for chunks. 9.00MiB in use in bin. 9.00MiB client-requested in use in bin.
2022-04-22 10:10:30.744021: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (16777216): 	Total Chunks: 0, Chunks in use: 0. 0B allocated for chunks. 0B in use in bin. 0B client-requested in use in bin.
2022-04-22 10:10:30.744046: I tensorflow/core/common_runtime/bfc_allocator.cc:1017] Bin (33

In [ ]:
def plot_fig(H,outdir):
    N=len(result.history['loss'])
    plt.style.use("ggplot")
    plt.figure(figsize=(10,6))
    plt.plot(np.arange(0, N), H.history["loss"], label="train_loss")
    plt.plot(np.arange(0, N), H.history["val_loss"], label="val_loss")
    plt.plot(np.arange(0, N), H.history["accuracy"], label="train_acc")
    plt.plot(np.arange(0, N), H.history["val_accuracy"], label="val_acc")
    plt.ylim(0,1)

    plt.title("Training Loss and Accuracy")
    plt.xlabel("Epoch #")
    plt.ylabel("Loss/Accuracy")
    plt.legend(loc="lower left")
    plt.savefig(outdir)
plot_fig(result,nickname+"_Loss_Acc_epoch.png")

In [ ]:
hdf5_path = '/home/nmic/input/nas5918/HJtest2-tf1/data/val-data-500.hdf5' 
fd = h5py.File(hdf5_path)
fd.keys()
xval,yval=np.asarray(fd['xval']),np.asarray(fd['yval'])
fd.close

In [ ]:
# model=load_model('/home/nmic/project/checkpoint/UNET_VGG-19e-val_loss0.918906.hdf5')

In [ ]:
i=0
n_label=6
nickname='unet_vgg'
result=model.predict(xval/255.0,)
pred=np.argmax(result,axis=-1)
newy=label_hot(yval,n_label)
val_plot_func(xval[i:i+20]/255.0,result[i:i+20,:,:,4],newy[i:i+20,:,:,4])
i+=30
#耕地=0，林地=1，草地=2，水域=3，居民地=4，未利用=5

In [ ]:
cal_table=val_cal(yval, pred, './'+nickname+'_result.csv', if_show=True)